<div dir="rtl" align="right">

# 📘 بصری‌سازی با PCA و انتخاب k در K-Means (Elbow & Silhouette) — جلسه 19/09/2025

در این نوت‌بوک، محتوای درس امروز را «گام‌به‌گام و عملی» گردآوری کرده‌ایم:
- ساخت دیتاست‌های مصنوعی (`make_blobs`, `make_moons`) برای تمرین.
- نرمال‌سازی با `StandardScaler` و چراییِ آن.
- تحلیل مؤلفه‌های اصلی (**PCA**) برای بصری‌سازی/کاهش‌بُعد و توضیح «واریانس توضیح‌داده‌شده».
- خوشه‌بندی با **K-Means** و انتخاب تعداد خوشه‌ها با دو معیار **Elbow (Inertia)** و **Silhouette**.
- فرهنگ واژگان (اصطلاحات فنی ↔ ترجمهٔ فارسی).
- «سؤالات تشریحی» برای سنجش فهم کامل درس.


</div>

In [ ]:

# کتابخانه‌ها
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# برای نمایش تمیزتر نمودارها
plt.rcParams['figure.figsize'] = (6, 4)


<div dir="rtl" align="right">

# 🎯 مرحله ۱: ساخت دیتاست‌های مصنوعی

اینجا دو دیتاست متفاوت می‌سازیم تا رفتار روش‌های خطی و غیرخطی را ببینیم:
- **Blobs**: خوشه‌های تقریباً کروی و جداشدنی؛ برای تمرین K-Means ایدئال است.
- **Moons**: دو هلال درهم‌تنیده با ساختار **غیرخطی**؛ جایی که K-Means و PCA خطی محدودیت دارند.


</div>

In [ ]:

# ساخت دیتاست‌ها (قابل تکرار با random_state)
X_blob, y_blob = make_blobs(
    n_samples=500, centers=4,
    cluster_std=[1.2, 1.0, 0.8, 1.1],
    random_state=42
)

X_moon, y_moon = make_moons(
    n_samples=500, noise=0.08, random_state=42
)

print('Shapes -> blob:', X_blob.shape, 'moons:', X_moon.shape)


<div dir="rtl" align="right">

# 🧭 مرحله ۲: نرمال‌سازی ویژگی‌ها با StandardScaler (چراییِ حیاتی)

**PCA** و **K-Means** هر دو مبتنی بر **فاصله**‌اند؛ بنابراین «مقیاس ویژگی‌ها» اثر مستقیم دارد.
با `StandardScaler` هر بُعد را به «میانگین صفر، انحراف معیار یک» می‌بریم تا همهٔ ابعاد **عادلانه** در محاسبات مشارکت کنند.


</div>

In [ ]:

# اسکیلِ مستقل برای هر دیتاست
scaler_blob = StandardScaler()
X_blob_s = scaler_blob.fit_transform(X_blob)

scaler_moon = StandardScaler()
X_moon_s = scaler_moon.fit_transform(X_moon)

X_blob_s[:3], X_moon_s[:3]  # پیش‌نمایش سه ردیف اول


<div dir="rtl" align="right">

# 🔎 مرحله ۳: PCA — ایده، کارکرد، واریانسِ توضیح‌داده‌شده

**PCA** محورهایی ارتوگونال می‌یابد که **بیشترین واریانس** داده روی آن‌هاست؛ سپس داده را روی این محورها تصویر می‌کند.
- چون هر دو دیتاست ما «دو-بعدی» هستند، `n_components=2` صرفاً یک **چرخش/تغییر مقیاس** ایجاد می‌کند (کاهش‌بُعد واقعی رخ نمی‌دهد).
- با این حال، برای بصری‌سازی، **همان ۲ مؤلفه** را رسم می‌کنیم و «نسبت واریانسِ توضیح‌داده‌شده» را نیز می‌بینیم.


</div>

In [ ]:

# PCA روی blob (دو مؤلفه)
pca_blob = PCA(n_components=2, random_state=42)
B2 = pca_blob.fit_transform(X_blob_s)

print('Explained variance ratio (blob):', pca_blob.explained_variance_ratio_,
      'sum =', pca_blob.explained_variance_ratio_.sum())

# نمودار PCA برای blob (یک نمودار در یک شکل)
plt.figure()
plt.scatter(B2[:, 0], B2[:, 1], s=10, c=y_blob)
plt.title('PCA Visualization — Blobs')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()


In [ ]:

# PCA روی moons (دو مؤلفه)
pca_moon = PCA(n_components=2, random_state=42)
M2 = pca_moon.fit_transform(X_moon_s)

print('Explained variance ratio (moons):', pca_moon.explained_variance_ratio_,
      'sum =', pca_moon.explained_variance_ratio_.sum())

# نمودار PCA برای moons (یک نمودار در یک شکل)
plt.figure()
plt.scatter(M2[:, 0], M2[:, 1], s=10, c=y_moon)
plt.title('PCA Visualization — Moons')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()


<div dir="rtl" align="right">

# 📊 مرحله ۴: K-Means و انتخاب k با Elbow و Silhouette

هدف: برای چند مقدار مختلفِ \(k\)، K-Means را اجرا کنیم و کیفیت خوشه‌بندی را با
- **Inertia (SSE)** برای روش **Elbow**
- **Silhouette** برای تعادل «چسبندگی درون‌خوشه‌ای/جدایی بین‌خوشه‌ای»
بسنجیم و بهترین \(k\) را انتخاب کنیم.


</div>

In [ ]:

def elbow_silhouette(X, kmin=2, kmax=10, random_state=42, n_init=10):
    """
    اجرای K-Means برای kهای مختلف و محاسبهٔ Inertia و Silhouette.
    خروجی: (لیست kها، لیست اینرسی‌ها، لیست سیلوئت‌ها)
    """
    ks = list(range(kmin, kmax + 1))
    inertia, sils = [], []
    for k in ks:
        kmeans = KMeans(n_clusters=k, n_init=n_init, random_state=random_state)
        labels = kmeans.fit_predict(X)
        inertia.append(kmeans.inertia_)
        sils.append(silhouette_score(X, labels))
    return ks, inertia, sils


In [ ]:

# اجرای معیارها روی blob
ks_b, inertia_b, sils_b = elbow_silhouette(X_blob_s, 2, 10)

# Elbow (inertia)
plt.figure()
plt.plot(ks_b, inertia_b, marker='o')
plt.xlabel('k'); plt.ylabel('Inertia (SSE)'); plt.title('Elbow — Blobs')
plt.show()

# Silhouette
plt.figure()
plt.plot(ks_b, sils_b, marker='s')
plt.xlabel('k'); plt.ylabel('Silhouette (mean)'); plt.title('Silhouette — Blobs')
plt.show()

best_k_sil = ks_b[int(np.argmax(sils_b))]
print('Best k by Silhouette (blobs):', best_k_sil)


In [ ]:

# اجرای معیارها روی moons (برای مشاهدهٔ محدودیت K-Means در ساختار غیرخطی)
ks_m, inertia_m, sils_m = elbow_silhouette(X_moon_s, 2, 10)

plt.figure()
plt.plot(ks_m, inertia_m, marker='o')
plt.xlabel('k'); plt.ylabel('Inertia (SSE)'); plt.title('Elbow — Moons')
plt.show()

plt.figure()
plt.plot(ks_m, sils_m, marker='s')
plt.xlabel('k'); plt.ylabel('Silhouette (mean)'); plt.title('Silhouette — Moons')
plt.show()

best_k_sil_m = ks_m[int(np.argmax(sils_m))]
print('Best k by Silhouette (moons):', best_k_sil_m)


<div dir="rtl" align="right">

# 🧾 فرهنگ واژگان (اصطلاحات فنی ↔ ترجمهٔ فارسی)

- **Dataset** = مجموعه‌داده  
- **Feature / Dimension** = ویژگی / بُعد  
- **Standardization** = استانداردسازی (میانگین صفر، انحراف معیار یک)  
- **PCA (Principal Component Analysis)** = تحلیل مؤلفه‌های اصلی  
- **Principal Component (PC)** = مؤلفهٔ اصلی  
- **Explained Variance Ratio** = نسبت واریانسِ توضیح‌داده‌شده  
- **Projection** = تصویر/پروژکشن روی فضا/محورها  
- **Clustering** = خوشه‌بندی  
- **K-Means** = خوشه‌بندی k-میانگین  
- **Centroid** = مرکز خوشه  
- **Inertia (SSE)** = اینرسی / مجموع مربعات خطا تا نزدیک‌ترین مرکز  
- **Elbow Method** = روش آرنج  
- **Silhouette Score** = امتیاز سیلوئت  
- **Within-Cluster Distance** = فاصلهٔ درون‌خوشه‌ای  
- **Between-Cluster Separation** = جدایی بین‌خوشه‌ای  
- **Nonlinear Structure** = ساختار غیرخطی  
- **Kernel Methods** = روش‌های هسته‌ای (برای نگاشت به فضای ویژگی با بُعد بالاتر)  


</div>

<div dir="rtl" align="right">

# 📝 سؤالات تشریحی (نشانهٔ فهم کامل)

1) چرا قبل از اجرای **PCA** و **K-Means** باید داده را استاندارد کنیم؟ دقیقاً چه مشکلی رخ می‌دهد اگر این کار را نکنیم؟  
2) در دادهٔ دو-بعدی، اجرای `PCA(n_components=2)` چه اثری دارد و چرا «کاهش‌بُعد واقعی» محسوب نمی‌شود؟  
3) تعریف دقیق امتیاز **Silhouette** برای یک نقطه چیست؟ رابطهٔ آن با \(a(i)\) و \(b(i)\) را بنویسید و تفسیر عددی آن را توضیح دهید.  
4) چرا روش **Elbow** همیشه کاهش اینرسی را نشان می‌دهد و چگونه «نقطهٔ آرنج» را تشخیص می‌دهیم؟ محدودیت‌های این روش چیست؟  
5) برای دیتاست **moons** چرا K-Means انتخاب مناسبی نیست؟ چه الگوریتم‌ها/روش‌های جایگزینی را پیشنهاد می‌کنید و چرا؟  
6) اگر در خروجی سیلوئت، بهترین \(k\) عدد بزرگی شد ولی یک خوشه فقط چند نقطه داشت، چه نتیجه‌ای می‌گیرید و چه اقداماتی پیشنهاد می‌شود؟  


</div>

<div dir="rtl" align="right">

# 📚 منابع جلسه

- **Hands-On Machine Learning (Géron)** — فصل «کاهش بُعد (PCA)» و فصل «یادگیری بدون ناظر: خوشه‌بندی (K-Means)»  
- **Pattern Recognition and Machine Learning (Bishop)** — فصل «Continuous Latent Variables» (بخش PCA)  
- **Scikit-learn User Guide** — بخش‌های `PCA`, `KMeans`, `silhouette_score`  
- **Effective Python (Slatkin)** — نکات بهترین‌عمل‌کرد در سازمان‌دهی کد و آزمایش‌پذیری  


</div>